In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt
# imports necessarios

In [ ]:
def get_data():
    num_sequences = 1000
    seq_length = 10
    x = np.random.rand(num_sequences, seq_length, 1)
    y = np.random.rand(num_sequences)
    return x, y


seq_length = 10 
num_sequences = 1000


x, y = get_data()
x_train, x_test = x[:800], x[800:]
y_train, y_test = y[:800], y[800:]


model = Sequential([ 
    LSTM(50, activation='relu', input_shape=(seq_length, 1)),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')



In [ ]:
history = model.fit(x_train, y_train, epochs=100, validation_data=(x_test, y_test))


In [ ]:
#visualizar treinamento
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()